# Neurosymbolic Integration: Enforcing Business Rules with Strands Hooks

Based on: [ATA: Autonomous Trustworthy Agents](https://arxiv.org/html/2510.16381v1)

## The Problem

AI agents hallucinate when business rules are expressed only in natural language prompts:

- **Parameter errors**: Agent calls `book_hotel(guests=15)` despite "Maximum 10 guests" in docstring
- **Completeness errors**: Agent executes bookings without required payment verification
- **Tool bypass behavior**: Agent confirms success without calling validation tools

**Why prompt engineering fails**: Prompts are suggestions, not constraints. Agents can ignore docstring instructions because they're processed as text, not executable rules.

## The Solution: Strands Hooks for Neurosymbolic Validation

Strands Agents provides **hooks**—a composable system that intercepts tool calls before execution to enforce symbolic rules:

```
User Query → LLM (understands) → Tool Selection → Hook (validates) → Execute or Block
```

**Key insight**: Use Strands hooks to validate tool calls before execution. The LLM cannot bypass rules enforced at the framework level.

## How It Works

1. **Define symbolic rules** - Business logic as verifiable code
2. **Create validation hook** - Intercepts tool calls with `BeforeToolCallEvent`
3. **Define clean tools** - No validation logic mixed in
4. **Attach hook to agent** - Framework handles the rest

## What We Test

We test 3 scenarios where agents commonly hallucinate:
- Confirming bookings without payment verification
- Booking hotels with too many guests (>10)
- Valid operations that should succeed

The hook blocks invalid operations before tools execute, preventing hallucinations.

In [ ]:
!pip install -r requirements.txt

## Configure API Key

Set your OpenAI API key. Get one at https://platform.openai.com/api-keys

In [ ]:
import os
# os.environ['OPENAI_API_KEY'] = 'your-key-here'  # Uncomment and set your key
assert os.getenv('OPENAI_API_KEY'), 'Set OPENAI_API_KEY env var or uncomment the line above'

## Setup

In [ ]:
from strands import Agent, tool
from strands.hooks import HookProvider, HookRegistry, BeforeToolCallEvent
from strands.models.openai import OpenAIModel
from datetime import datetime
from rules import BOOKING_RULES, CONFIRMATION_RULES, validate

# Default: OpenAI
MODEL = OpenAIModel(model_id="gpt-4o-mini")

# Alternative: Bedrock (uncomment to use)
# MODEL = "us.anthropic.claude-3-haiku-20240307-v1:0"

# Simulated state
STATE = {
    "bookings": {"BK001": {"hotel": "Grand Hotel", "check_in": "2026-02-15", "guests": 2}},
    "payments": {}  # No payments verified yet
}

print("✅ Setup complete")

## Step 1: Define Symbolic Rules

Business rules as verifiable code. These rules cannot be bypassed by prompt engineering.

In [ ]:
from rules import BOOKING_RULES, CONFIRMATION_RULES

print("Booking Rules:")
for rule in BOOKING_RULES:
    print(f"  - {rule.name}: {rule.message}")

print("\nConfirmation Rules:")
for rule in CONFIRMATION_RULES:
    print(f"  - {rule.name}: {rule.message}")

## Step 2: Create Validation Hook

The hook intercepts tool calls before execution using `BeforeToolCallEvent`. If rules are violated, it cancels the tool with `event.cancel_tool`.

In [ ]:
class NeurosymbolicHook(HookProvider):
    """Validates tool calls against symbolic rules before execution"""
    
    def __init__(self, state: dict):
        self.state = state
        self.rules = {
            "book_hotel": BOOKING_RULES,
            "confirm_booking": CONFIRMATION_RULES,
        }
    
    def register_hooks(self, registry: HookRegistry) -> None:
        registry.add_callback(BeforeToolCallEvent, self.validate)
    
    def validate(self, event: BeforeToolCallEvent) -> None:
        tool_name = event.tool_use["name"]
        if tool_name not in self.rules:
            return
        
        ctx = self._build_context(tool_name, event.tool_use["input"])
        passed, violations = validate(self.rules[tool_name], ctx)
        
        if not passed:
            event.cancel_tool = f"BLOCKED: {', '.join(violations)}"
    
    def _build_context(self, tool_name: str, params: dict) -> dict:
        if tool_name == "book_hotel":
            try:
                ci = datetime.strptime(params["check_in"], "%Y-%m-%d")
                co = datetime.strptime(params["check_out"], "%Y-%m-%d")
                return {
                    "check_in": ci,
                    "check_out": co,
                    "guests": params.get("guests", 1),
                    "days_until_checkin": (ci - datetime.now()).days
                }
            except (ValueError, KeyError):
                # Return context that fails validation
                return {
                    "check_in": None,
                    "check_out": None,
                    "guests": 999,
                    "days_until_checkin": -999
                }
        elif tool_name == "confirm_booking":
            return {"payment_verified": params["booking_id"] in self.state["payments"]}
        return {}

print("✅ Hook created")

## Step 3: Define Clean Tools

Tools have no validation logic—they're simple and focused on business operations. The hook handles all validation.

In [ ]:
@tool
def book_hotel(hotel: str, check_in: str, check_out: str, guests: int = 1) -> str:
    """Book a hotel room."""
    return f"SUCCESS: Booked {hotel} for {guests} guests, {check_in} to {check_out}"

@tool
def process_payment(amount: float, booking_id: str) -> str:
    """Process payment for a booking."""
    if booking_id not in STATE["bookings"]:
        return "ERROR: Booking not found"
    STATE["payments"][booking_id] = amount
    return f"SUCCESS: Processed ${amount} for {booking_id}"

@tool
def confirm_booking(booking_id: str) -> str:
    """Confirm a booking."""
    return f"SUCCESS: Confirmed {booking_id}"

print("✅ Tools defined")

## Step 4: Create Agent with Hook

Attach the hook to the agent. Strands automatically intercepts tool calls and validates them before execution.

In [ ]:
hook = NeurosymbolicHook(STATE)
agent = Agent(
    tools=[book_hotel, process_payment, confirm_booking],
    hooks=[hook],
    model=MODEL
)

print("✅ Agent created with neurosymbolic hook")

## Test 1: Confirm Booking Without Payment

**Expected**: Hook blocks because payment is not verified

In [ ]:
result = agent("Confirm booking BK001")
print(result)

## Test 2: Book Hotel with Too Many Guests

**Expected**: Hook blocks because maximum is 10 guests

In [ ]:
result = agent("Book Grand Hotel for 15 people from 2026-03-20 to 2026-03-25")
print(result)

## Test 3: Valid Booking

**Expected**: Hook allows execution because all rules pass

In [ ]:
result = agent("Book Grand Hotel for 5 guests from 2026-03-20 to 2026-03-25")
print(result)

## Results

**Key Findings:**

✅ **Invalid operations blocked**: Hook prevents execution before tools run  
✅ **Valid operations succeed**: Rules don't interfere with legitimate requests  
✅ **Clean separation**: Tools have no validation logic  
✅ **LLM cannot bypass**: Rules enforced at framework level  

## Why Strands Hooks Excel

- **Simple integration**: `Agent(tools=[...], hooks=[hook])`
- **Centralized validation**: One hook validates all tools
- **Type-safe**: Strongly-typed event objects
- **Composable**: Multiple hooks can work together

## Conclusion

Strands hooks make neurosymbolic integration effortless. Define your rules, create a hook, attach it to your agent—the framework handles the rest. The LLM receives cancellation messages it cannot override, ensuring 100% rule compliance.